# Data Preparation

We need to represent and access info for all passengers: gender(weight), which group they belong to, if they have to transfert. We created a class Passenger to represent this data. We also created a class Group to reprenent each group (numer of women, men, transit time).

The main decision variable here is the allocation A : binary variable that says for each passenger where he is located : 

A[x, y, p.passenger_id]

For instance, if Passenger 1 is at the place : width=2, height=3, then allocation[2,3,1]=1. Else, it equals 0.

There are also the satisfaction variables: depends on the number of neighbors of the same group you have [to be completed]

In [2]:
class Passenger:
    def __init__(self, group_index, passenger_id, description, transit_time=0):
        """
            group_index : 
            passenger_id : 
            description : men, women, wheelchair, child
            business_economic : class (Business, Economic)
            transit_time : optional parameter (inf if there is not transit time)
        
        """
        self.group_index = group_index
        self.passenger_id = passenger_id
        self.description = description
        # self.business_economic = business_economic
        self.transit_time = transit_time
        self.weight()
    
    def weight(self):
        if self.description == 'women':
            self.weight = 70
        elif self.description == 'men':
            self.weight = 85
        elif self.description == 'child':
            self.weight = 35
        else: 
            self.weight = 70 # need to check this value

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
passenger_id = 0

In [5]:
class Group:

    def __init__(self, id_group, num_women, num_men, transit_time=0):
        self.id_group = id_group
        self.num_women = num_women
        self.num_men = num_men
        self.transit_time = transit_time

        self.members = []

        global passenger_id

        for i in range(self.num_women):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'women',
                self.transit_time
            ))
            passenger_id+=1
        
        for i in range(self.num_men):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'men',
                self.transit_time
            ))
            passenger_id+=1
        
        return
    
    
        

In [6]:
def read_and_preprocess(file, date):
    df = pd.read_excel(file, date)
    
    df['TransitTime (min)'] = df['TransitTime'].apply(lambda x: float(str(x).split(':')[0])*60+float(str(x).split(':')[1]))

    df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']] = df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']].fillna(0)

    return df

def get_list_passengers(df):
    list_groups = {}

    for idx, row in df.iterrows():

        list_groups[int(idx)] = Group(
            id_group=int(idx),
            num_women=int(row['Femmes']),
            num_men=int(row['Hommes']),
            transit_time=row['TransitTime (min)']
        )
    
    return list_groups

In [7]:
df = read_and_preprocess('DataSeating - V3.xls', '21Oct')

In [8]:
df.head()

,Numéro du groupe,Femmes,Hommes,Enfants,WCHR,WCHB,Classe,TransitTime,TransitTime (min)
0,1,0.0,1.0,0.0,0.0,0.0,Y,01:15:00,75.0
1,2,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
2,3,0.0,1.0,0.0,0.0,0.0,Y,00:00:00,0.0
3,4,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
4,5,0.0,2.0,0.0,0.0,0.0,Y,05:25:00,325.0


In [9]:
groups_list = get_list_passengers(df)

In [10]:
passengers_list = list()

for g in groups_list.keys():
    passengers_list += groups_list[g].members

In [11]:
len(passengers_list)

151

In [12]:
passengers_list

# Creating the Airplane

In [24]:
airplane = dict()

airplane['xmax'] = 28
airplane['ymax'] = 7
airplane['corridor_y'] = 4
airplane['max_transition_row'] = 13

airplane['seats'] = dict()

# considering there is only business class
real_seats = []
corridor_seats = []
transition_seats = []

for x in range(1, airplane['xmax']+1):
    for y  in range(1, airplane['ymax']+1):
        if y != airplane['corridor_y']:
            real_seats.append([x, y])
        if y == airplane['corridor_y']:
            corridor_seats.append([x, y])
        if x <= airplane['max_transition_row'] and y != airplane['corridor_y']:
            transition_seats.append([x, y])

airplane['seats']['real'] = real_seats
airplane['seats']['corridor'] = corridor_seats
airplane['seats']['transition_possible_seats'] = transition_seats

del real_seats, corridor_seats, transition_seats

print(airplane)



{'xmax': 28, 'ymax': 7, 'corridor_y': 4, 'max_transition_row': 13, 'seats': {'real': [[1, 1], [1, 2], [1, 3], [1, 5], [1, 6], [1, 7], [2, 1], [2, 2], [2, 3], [2, 5], [2, 6], [2, 7], [3, 1], [3, 2], [3, 3], [3, 5], [3, 6], [3, 7], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [4, 7], [5, 1], [5, 2], [5, 3], [5, 5], [5, 6], [5, 7], [6, 1], [6, 2], [6, 3], [6, 5], [6, 6], [6, 7], [7, 1], [7, 2], [7, 3], [7, 5], [7, 6], [7, 7], [8, 1], [8, 2], [8, 3], [8, 5], [8, 6], [8, 7], [9, 1], [9, 2], [9, 3], [9, 5], [9, 6], [9, 7], [10, 1], [10, 2], [10, 3], [10, 5], [10, 6], [10, 7], [11, 1], [11, 2], [11, 3], [11, 5], [11, 6], [11, 7], [12, 1], [12, 2], [12, 3], [12, 5], [12, 6], [12, 7], [13, 1], [13, 2], [13, 3], [13, 5], [13, 6], [13, 7], [14, 1], [14, 2], [14, 3], [14, 5], [14, 6], [14, 7], [15, 1], [15, 2], [15, 3], [15, 5], [15, 6], [15, 7], [16, 1], [16, 2], [16, 3], [16, 5], [16, 6], [16, 7], [17, 1], [17, 2], [17, 3], [17, 5], [17, 6], [17, 7], [18, 1], [18, 2], [18, 3], [18, 5], [18, 6], [18, 

# Defining the model

In [14]:
from gurobipy import *

In [15]:
m = Model('First Try')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-26


## Decision Variables

In [16]:
A = dict()

for x in range(airplane['xmax']+1):
    for y in range(airplane['ymax']+1):
        for p in passengers_list:
            A[x, y, p.passenger_id] = m.addVar(vtype=GRB.BINARY, name="A_{},{},{}".format(x,y,p.passenger_id))

## Constraints

### Barycenter

Separate the X and Y axis. Then separate each axis into 2 constraints: 1 lower constraint, 1 upper constraint, to fit into the defined barycenter area : 13<X<17 and 3<Y<5. To calculate the barycenter, we multipliy for each person its weight by its distance to the center of the airplane, multiplied by the allocation. The sum is then divided by the total weight (of every passengers). 

$$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*x  /total_{weight} \leq  1$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*x  /total_{weight} \geq -1$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*y  /total_{weight} \leq 2$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*y  /total_{weight} \geq -2$$ for i in range(width) for j in range(height)

With x being the distance from the passenger to the center on the X axis and y being the distance from the passenger to the center on the Y axis


In [17]:
# Barycenter: x \in [13,17], y \in [3,5]
total_weight_passengers = sum(p.weight for p in passengers_list)

barycenter = dict()

barycenter['xinf'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 13*total_weight_passengers, name="barycenter_x_inf")
barycenter['xsup'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 17*total_weight_passengers, name="barycenter_x_sup")
barycenter['yinf'] = m.addConstr(quicksum(p.weight*y*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 3*total_weight_passengers, name="barycenter_y_inf")
barycenter['ysup'] = m.addConstr(quicksum(p.weight*y*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 5*total_weight_passengers, name="barycenter_y_sup")


### No passenger in the corridor

In [18]:
corridor = m.addConstr(quicksum(A[x,y,p.passenger_id] for x, y in airplane['seats']['corridor'] for p in passengers_list) == 0)

### At most 1 person per seat constraint: in the plane, there can't be more than 1 person in each seat.

$$\sum_{passenger = 1}^{p}allocation[p.passenger_id,i,j]  \leq 1 $$for i in range(width) for j in range(height)

p being the number of passengers

In [19]:
passenger_seat_constr = dict()

for x, y in airplane['seats']['real']:
    passenger_seat_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for p in passengers_list) <= 1, name='1 passenger at most at seat {},{}'.format(x,y))


### One seat per passenger

Everyone has 1 and only 1 seat constraint: each passenger has to be seated in only one place.

$$\sum_{i,j = 1}^{w,h}allocation[p.passenger_id,i,j] =1 $$

for p in passengers

With w the width (X), h the height (Y)

In [20]:
seat_passenger_constr = dict()

for x,y in airplane['seats']['real']:
    seat_passenger_constr[(x, y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for p in passengers_list) == 1, name='1 seat per passenger {}'.format(p.passenger_id))

### Passengers in transition

Constraint of transfert passenger : if a passenger has to transfert, he needs to be in approximately the first third of the plane (X axis). We chose row 13, to be a bit more flexible. No constraint on the Y axis. If there is no transfert, the passenger can be anywhere.


 For p in passengers,  $$  p.transfert=1 \rightarrow allocation[p.passenger_{id}][x] \leq 13 \;\;\;\;  \;\;\;$$

This will be implemented with an if condition before the constraint creation.

In [25]:
transition_constr = dict()

for p in passengers_list:
    if p.transit_time > 0:
        transition_constr[(p.passenger_id, x, y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for x, y in airplane['seats']['transition_possible_seats'] for p in passengers_list) == 1, name='passenger {} in transition'.format(p.passenger_id))